In [ ]:
#Import libraries
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt

## Data reading and wrangling

In [ ]:
#set directory for results
res = 'run_v1/csv_results/'

In [ ]:
#read parquet file with aggregated results
df = pd.read_parquet(res + 'all.parquet')

In [ ]:
#create new columns

df['route_id'] = df['source'].str.split('_').str[1]
df['route_id'] = df['source'].str.split('_').str[1]
df['time_day'] = df['source'].str.split('_').str[2]
df['type'] = df['source'].str.split('_').str[3]
df

In [ ]:
#create dataset with aggregated distance
dfw = df.groupby('source')['distance'].sum()

dfw = dfw.to_frame().reset_index()



In [ ]:
#create dataframe with relevant variables

df_exp = df[['source', 's_exp', 'time_day']]
df_exp = df_exp.drop_duplicates()

In [ ]:
#merge dataframe
df_merged = dfw.merge(df_exp[['source', 's_exp', 'time_day']], on='source', how='inner')

df_merged['route_type'] = df_merged['source'].str.split('_').str[3]

df_merged

## Length and solar exposure distribution

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
#first plot
palette = sns.color_palette("Set2", 2)

p = sns.catplot(
    kind='box',
    data=df_merged,
    x='route_type',
    y='s_exp',
    hue='route_type',
    legend=False,
    col='time_day',
    col_wrap=4,
    height=6,
    aspect=0.8,
    palette=palette,
    col_order=['morning', 'noon', 'afternoon', 'evening']
)


p.set_titles("{col_name}")
p.set_axis_labels("", "Solar Exposure [%]")
p.despine(left=True)
p.fig.subplots_adjust(top=0.9)
p.fig.suptitle("Solar Exposure Distribution by Route Type and Time of Day", fontsize=10, fontweight='bold')


p.fig.text(0.5, 0.02, "Route type", ha='center', fontsize=12)

p.savefig("s_exp_route_type_timeofday.png", dpi=400, bbox_inches='tight')

plt.show()

In [ ]:
#second plot

palette = sns.color_palette("Set2", 2)

p = sns.catplot(
    kind='box',
    data=df_merged,
    x='route_type',
    y='distance',
    hue='route_type',
    legend=False,
    col='time_day',
    col_wrap=4,
    height=6,
    aspect=0.8,
    palette=palette,
    col_order=['morning', 'noon', 'afternoon', 'evening']
)


p.set_titles("{col_name}")
p.set_axis_labels("", "Length [m]")
p.despine(left=True)
p.fig.subplots_adjust(top=0.9)
p.fig.suptitle("Length Distribution by Route Type and Time of Day", fontsize=10, fontweight='bold')


p.fig.text(0.5, 0.02, "Route type", ha='center', fontsize=12)

p.savefig("length_route_type_timeofday.png", dpi=400, bbox_inches='tight')

plt.show()

## Find biggest length difference

In [ ]:
df_merged['source_no_route'] = df_merged['source'].str.rsplit('_', n=1).str[0]
df_merged

In [ ]:
wide_df = df_merged.pivot(
    index='source_no_route',
    columns='route_type',
    values='distance'
)

In [ ]:
wide_df['Score_diff'] = wide_df['recommended'] - wide_df['shortest']
wide_df

In [ ]:
big_diff = (wide_df['recommended'] - wide_df['shortest']).abs().idxmax()
big_diff

In [ ]:
#plot with folium
gdf1 = gpd.read_file(f'../data/run_v1/01_raw/01_routes/{big_diff}_recommended.geojson')
gdf2= gpd.read_file(f'../data/run_v1/01_raw/01_routes/{big_diff}_shortest.geojson')

df1 = df_merged[df_merged['source'] == f'{big_diff}_recommended']
df2 = df_merged[df_merged['source'] == f'{big_diff}_shortest']

gdf1['solar_exp'] = df1.iloc[0]['s_exp']
gdf1['source'] = df1.iloc[0]['source']
gdf2['solar_exp'] = df2.iloc[0]['s_exp']
gdf2['source'] = df2.iloc[0]['source']


gdf_combined = pd.concat([gdf1, gdf2], ignore_index=True)

gdf_combined


In [ ]:
import folium
import branca

m = folium.Map([49.4, 8.7], zoom_start=12, tiles='cartodbpositron')

min_val = gdf_combined['solar_exp'].min()
max_val = gdf_combined['solar_exp'].max()
colormap = branca.colormap.LinearColormap(
    colors=['blue', 'green', 'yellow', 'red'],
    vmin=min_val,
    vmax=max_val,
    caption='Solar Exposure'
)



for source in gdf_combined['source'].unique():
    fg = folium.FeatureGroup(name=f"Route: {source}")
    
    geojson = folium.GeoJson(
        gdf_combined[gdf_combined['source'] == source],
        style_function=lambda feature, cmap=colormap: {
            'color': cmap(feature['properties']['solar_exp']),
            'weight': 3,
            'opacity': 1
        }
    )
    
    geojson.add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

colormap.add_to(m)

m


## Find biggest Solar exposure difference

In [ ]:
wide_df = df_merged.pivot(
    index='source_no_route',
    columns='route_type',
    values='s_exp'
)

In [ ]:
big_diff = (wide_df['recommended'] - wide_df['shortest']).abs().idxmax()
big_diff

In [ ]:
gdf1 = gpd.read_file(f'../data/run_v1/01_raw/01_routes/{big_diff}_recommended.geojson')
gdf2= gpd.read_file(f'../data/run_v1/01_raw/01_routes/{big_diff}_shortest.geojson')

df1 = df_merged[df_merged['source'] == f'{big_diff}_recommended']
df2 = df_merged[df_merged['source'] == f'{big_diff}_shortest']

gdf1['solar_exp'] = df1.iloc[0]['s_exp']
gdf1['source'] = df1.iloc[0]['source']
gdf2['solar_exp'] = df2.iloc[0]['s_exp']
gdf2['source'] = df2.iloc[0]['source']


gdf_combined = pd.concat([gdf1, gdf2], ignore_index=True)

gdf_combined

In [ ]:
import folium
import branca

m = folium.Map([49.4, 8.7], zoom_start=12, tiles='cartodbpositron')

min_val = gdf_combined['solar_exp'].min()
max_val = gdf_combined['solar_exp'].max()
colormap = branca.colormap.LinearColormap(
    colors=['blue', 'green', 'yellow', 'red'],
    vmin=min_val,
    vmax=max_val,
    caption='Solar Exposure'
)



for source in gdf_combined['source'].unique():
    fg = folium.FeatureGroup(name=f"Route: {source}")

    geojson = folium.GeoJson(
        gdf_combined[gdf_combined['source'] == source],
        style_function=lambda feature, cmap=colormap: {
            'color': cmap(feature['properties']['solar_exp']),
            'weight': 3,
            'opacity': 1
        }
    )

    geojson.add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

colormap.add_to(m)

m

## Correlation Matrix


In [ ]:
distance = df_merged["distance"]
sol_exp = df_merged["s_exp"]

plt.plot(distance, sol_exp, "o")
plt.xlabel("distance")
plt.ylabel("sol_exp")
plt.show()

In [ ]:
x = distance.copy()
y = sol_exp.copy()
x_bar = np.mean(distance)
y_bar = np.mean(sol_exp)

np.sum((x - x_bar) * (y - y_bar)) / (
    np.sqrt(np.sum((x - x_bar) ** 2)) * np.sqrt(sum((y - y_bar) ** 2))
)


(
    np.cov(x, y) / (np.std(x) * np.std(y))
)



print(np.corrcoef(x, y)[0, 1])
